# Simplified Contour Trees with TTK
(with ray tracing)

In [1]:
from paraview.simple import *
from ipyparaview.widgets import PVDisplay

First, let's load a geometry model and compute its height function.

In [2]:
# load a geometry model
model = XMLUnstructuredGridReader(FileName=['/data/dragon.vtu'])

# run an elevation filter
elevation = Elevation(Input=model)
elevation.LowPoint =  [0, -50, 0]
elevation.HighPoint = [0,  50, 0]

Next, create a view to render the model only. For additional fanciness, turn on ray tracing and shadows.

In [3]:
def make_view():
    '''helper function to create and configure a view'''
    view = CreateView('RenderView')
    view.ViewSize = (600, 400)
    view.Background = [1, 1, 1]
    view.EnableRayTracing = True
    view.Shadows = True
    view.EnableRenderOnInteraction = False
    
    return view

view = make_view()
    
Show(elevation, view)
ResetCamera(view)
display(PVDisplay(view))

PVDisplay(resolution=(600, 400))

In [4]:
# compute the persistence diagram ...
persdiag = TTKPersistenceDiagram(Input=elevation)
persdiag.ScalarField = ['POINTS', 'Elevation']
persdiag.InputOffsetField = ['POINTS', 'Elevation']
persdiag.EmbedinDomain = 1

# ... and filter non-persistent pairs ...
threshold = Threshold(Input=persdiag)
threshold.Scalars = ['CELLS', 'Persistence']
threshold.ThresholdRange = [0.1, 1.0]

# which serve as constaint to simplify the function
simplified = TTKTopologicalSimplification( Domain=elevation, Constraints=threshold )
simplified.ScalarField = ['POINTS', 'Elevation']
simplified.InputOffsetField = ['POINTS', 'Elevation']
simplified.VertexIdentifierField = ['POINTS', 'Birth']
simplified.OutputOffsetScalarField = ''

# finally, compute the contour tree of the simplified function
mctftm = TTKMergeandContourTreeFTM(Input=simplified)
mctftm.ScalarField = ['POINTS', 'Elevation']
mctftm.InputOffsetField = ['POINTS', 'Elevation']

# turn critical points (contour tree vertices) into spheres
# for better visibility
critpts = TTKIcospheresFromPoints(Input=mctftm)
critpts.Subdivisions = 2
critpts.Radius = 5.0

In [ ]:
# create a second view to show the simplified tree
view2 = CreateView('RenderView')
view2.ViewSize = (800,300)
view2.EnableRayTracing = True
view2.Shadows = True

Show(critpts, view2)
Show(threshold, view2)
Show(model, view2).Opacity = 0.2

ResetCamera(view2)

disp = PVDisplay(view2)
display(disp)

In [ ]:
# Interact from ipywidgets offers us a simple way to interactively control values with a callback function
from ipywidgets import interact

# sets the isosurface value, triggers a pipeline update, and renders the result
@interact(min=0.0, max=1.0)
def setThreshold(thresh=0.1):
    threshold.ThresholdRange = [thresh, 1.0]